In [22]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import math

## fill up the NaN

In [23]:
np.random.seed(2)
# After reading the csv file, first we need to fill up the NaN by some method. df_filled is the final filled dataframe, which we work on further.
df = pd.read_csv('final_cleaned_data.csv')
df_new = df[['BUS5','NEWS1','UNEMP','PX1Q1','NUMKID','EGRADE',
            'HOM','CAR','INCOME','VEHNUM','PSSA','PCRY','GAS1']]
df_new['BUS5'] = df_new['BUS5']/5
df_new['NEWS1'] = df_new['NEWS1']/10
df_new['EGRADE'] = df_new['EGRADE']/5
df_new['INCOME'] = df_new['INCOME']/10000
df_new['PSSA'] = df_new['PSSA']/10
df_new['GAS1'] = df_new['GAS1']/10
imputer = KNNImputer(n_neighbors=2)
df_filled = pd.DataFrame(imputer.fit_transform(df_new), columns=df_new.columns)
df_filled['BUS5'] = df_filled['BUS5']*5
df_filled['NEWS1'] = df_filled['NEWS1']*10
df_filled['EGRADE'] = df_filled['EGRADE']*5
df_filled['INCOME'] = df_filled['INCOME']*10000
df_filled['PSSA'] = df_filled['PSSA']*10
df_filled['GAS1'] = df_filled['GAS1']*10
df_filled = df_filled.join(df['YYYYQ'])
df_filled = df_filled.join(df['gdp_increase_rate'])

<ipython-input-23-d0a0d9f92508>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['BUS5'] = df_new['BUS5']/5
<ipython-input-23-d0a0d9f92508>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['NEWS1'] = df_new['NEWS1']/10
<ipython-input-23-d0a0d9f92508>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

## load data

In [24]:
data = df_filled.dropna()

In [25]:
# get the X and Y
X = data.iloc[:, :-2]
Y = data['gdp_increase_rate']

# standardize the x variables
for col in X.columns:
    X[col] = stats.zscore(X[col])

In [26]:
# split the dataset into train set and test set for validation.
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)


## model training


In [27]:
# function to choose the best params for each model
# input potential param candidate list, mse_list and r2_list.
def para_selection(para_list, mse_list, r2_list):
    df = pd.DataFrame([para_list, mse_list, r2_list], index = ['parameter', 'mse', 'r2'])
    df = df.T
    df_selected = df.loc[df['mse'] == min(df['mse'])]
    print(df_selected)
    para = df_selected.iloc[0, 0]
    mse = df_selected.iloc[0, 1]
    r2 = df_selected.iloc[0, 2]
    return para, mse, r2

In [28]:
# function to predict
# input: the "fitted" model, and the test set of x
def ml_prediction(model, x_test):
    y_test_pred = model.predict(x_test)
    return y_test_pred

### Ridge


In [29]:
mse_list = []
r2_list = []
alphas = np.arange(0.01, 1.01, 0.01) # the potential candidate for the params
for alpha in alphas:
    ridge = Ridge(alpha = alpha)
    ridge.fit(x_train, y_train)
    y_test_pred = ridge.predict(x_test)
    mse_list.append(mean_squared_error(y_test, y_test_pred))
    r2_list.append(r2_score(y_test, y_test_pred))

In [30]:
alpha, mse, r2 = para_selection(alphas, mse_list, r2_list) # here alpha is the best params
ridge_model = Ridge(alpha = alpha) # set up the model (but does not fit yet, the fitting is at the below)

    parameter       mse        r2
99        1.0  0.000043  0.393953


### random forest


In [43]:
mse_list = []
r2_list = []
n_est = range(10, 100)
for n in n_est:
    rf = RandomForestRegressor(n_estimators = n, random_state = 1)
    rf.fit(x_train, y_train)
    y_test_pred = rf.predict(x_test)
    mse_list.append(mean_squared_error(y_test, y_test_pred))
    r2_list.append(r2_score(y_test, y_test_pred))

plt.plot(n_est, mse_list)
plt.title("Random Forest Model")
plt.ylabel('MSE',fontsize=14)
plt.xlabel('Number of Estimators',fontsize=14)
plt.savefig("rf_new.png",pad_inches=0.2,dpi=200,bbox_inches='tight')
plt.clf()
# the plot below is the MSE v.s. n_estimators, from which we pick the param that minimizes the MSE.

<Figure size 432x288 with 0 Axes>

In [32]:
n, mse, r2 = para_selection(n_est, mse_list, r2_list)
rf_model = RandomForestRegressor(n_estimators = int(n), random_state = 1)
# again, set up the model, but not fitted.

   parameter       mse        r2
2       12.0  0.000038  0.462958


### XGBoost


In [42]:
mse_list = []
r2_list = []
etas = np.linspace(0.1, 1.5, 200)
for eta in etas:
    xgb = XGBRegressor(eta = eta, n_estimators = 100, random_state = 1)
    xgb.fit(x_train, y_train)
    y_test_pred = xgb.predict(x_test)
    mse_list.append(mean_squared_error(y_test, y_test_pred))
    r2_list.append(r2_score(y_test, y_test_pred))

# the plot below is the MSE v.s. etas, from which we pick the param that minimizes the MSE.
plt.plot(etas, mse_list)
plt.title("XG BOOST Model")
plt.xlabel('ETAs',fontsize=14)
plt.ylabel('MSE',fontsize=14)

plt.savefig("xg_new.png",dpi=200,bbox_inches='tight')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [34]:
eta, mse, r2 = para_selection(etas, mse_list, r2_list)
xgb_model = XGBRegressor(eta = eta, n_estimators = 100, random_state = 1)

    parameter       mse        r2
23   0.261809  0.000033  0.534914


From the training and validation above, xgboost is the best model and we use it for future prediction.

## Prediction

In [35]:
# Here is the fitting action, must run before calling the predicting() function.
xgb_model.fit(X,Y)
# rf_model.fit(X,Y)
# ridge_model.fit(X,Y)

# Here get the x variables
df2 = df_filled.loc[:, ~df_filled.columns.isin(['gdp_increase_rate','YYYYQ'])]


# function to get the mean of the change rate of the column
def get_mu(col):
    rate = col.pct_change()
    rate.pop(0)
    mu = np.nanmean(rate)
    return mu

# function to get the std of the change rate of the column
def get_std(col):
    rate = col.pct_change()
    rate.pop(0)
    std = np.nanstd(rate)
    return std


In [36]:
# return the times we need to predict by receiving the string time stamp input
# input: yearq is string of the time stamp, like '20234' for 4th quarter of 2023.
# predicts: the pre-generated predictions for future quarters
def predict(yearq,predicts):
    quarter = int(yearq)
    year = quarter//10
    quarter = quarter%10
    if year==2023:
        times = quarter-1
    else:
        times = (year-2024)*4+3+quarter
    return predicts[times-1]


# df_tmp is the dataframe of the filled data
# model is the fitted input ML model
def predicting_gen(df_tmp,model,max_quarter=12):
    predictions = []
    mus = df_tmp.apply(get_mu,axis=0)
    stds = df_tmp.apply(get_std,axis=0)
    res = df_tmp.values[-1].tolist()
    for i in range(max_quarter):
        for j in range(len(mus)):
            tmp = res[j]
            res[j] = tmp * (1+np.random.normal(mus[j],stds[j]))
            if math.isnan(res[j]):
                res[j] = tmp
        res_df = pd.DataFrame(res, index=['BUS5','NEWS1','UNEMP','PX1Q1','NUMKID','EGRADE',
                'HOM','CAR','INCOME','VEHNUM','PSSA','PCRY','GAS1']).transpose()
        df_all = pd.concat([df_tmp,res_df],axis=0,join='inner')
        for col in df_all.columns:
            df_all[col] = stats.zscore(df_all[col])
        predictions.append(ml_prediction(model,df_all.tail(1))[0])
    return predictions

In [37]:
# call the predicting_gen() to get the predicted results for maximum of 12quarters in the future , df2 is the original filled x variables dataframe, xgb_model: fitted models.
# predict() return result is the predicted gdp change rate of the quarter.
preds = predicting_gen(df2,xgb_model)
print(preds)


[0.0038189576, 0.0035301175, 0.0017585831, 0.0024566867, 0.0041350494, 0.016022118, 0.012911082, 0.015083434, 0.018932827, -0.008756744, 0.0047494387, -0.08710402]


/Users/hatj/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1539: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe')


In [38]:
dict_preds = {
    "20232":preds[0],
    "20233":preds[1],
    "20234":preds[2],
    "20241":preds[3],
    "20242":preds[4],
    "20243":preds[5],
    "20244":preds[6],
    "20251":preds[7],
    "20252":preds[8],
    "20253":preds[9],
    "20254":preds[10],
    "20261":preds[11]    
}
def predict1(yearq):
    if yearq in dict_preds.keys():
        return dict_preds[yearq]
    else:
        raise(ValueError)

In [39]:
print(predict1('20241')==predict('20241',preds))

True
